In [ ]:
import pandas as pd
import re

FILE = 'salix008_tab.txt'

with open(FILE,'r') as f:
    out = f.read()
    out = re.sub("(  +)", ",", out)
    out = re.sub("\*\*\*\*\*\*\*", "", out)
    with open(f"{FILE}_processed",'w') as o:
        o.write(out)

salix_df = pd.read_csv(f"{FILE}_processed")
wheatfen_df = pd.read_excel("/home/markhaoxiang/Projects/salix/Wheatfen_treecores.xlsx")

wheatfen_df_diameter = wheatfen_df[['ID', 'Diameter/cm']].dropna()
wheatfen_df_diameter['ID'] = wheatfen_df_diameter['ID'].apply(lambda x : x.replace("?",""))

In [ ]:
salix_df

In [ ]:
for wfp in salix_df.columns[0:]:
    d = wheatfen_df_diameter[(wheatfen_df_diameter['ID']==wfp[:-1])]['Diameter/cm'].values
    # Data not available
    if len(d) != 1:
        continue
    d = d[0]
    results = [d]
    for year in range(2022,1986,-1):
        x = salix_df[(salix_df['year']==year)][wfp].values[0]
        d = d - x*2/10
        results.append(round(d,5))
    results.reverse()
    salix_df[f"{wfp}_diameter"] = pd.Series(results)
    

In [ ]:
salix_df